### Functions to prompt LLM

This function generates a prompt for the LLM with a demographic profile of a survey respondents and a description of the two scenarios that the respondents encountered during the experiments. 

The demographic profile on the age, education, gender, and income that survey respondents reported after the Moral Machine experiment. The order in which these characteristics appear is randomized. It was up to the respondents to decide whether to take the survey or not.

The description of the scenarios is generated from the data matrix. One dilemma consists of two scenarios, presented side by side during the experiment. If respondents click on the left side, they opt for the death of the characters represented in that outcome while the characters on the right side will be saved (and vice versa). 

To generate the descriptions of the dilemmas, we extended code written by Takemoto (2024). This study generates new dilemmas by randomly combining features of scenarios (e.g. the composition of characters) and prompts LLMs to evaluate these dilemmas. In contrast, our study takes the existing dilemmas from Awad et al. (2018) and prompts LLMs to predict how survey respondents evluated the dilemmas.

In [14]:
from collections import Counter
from itertools import repeat
import pandas as pd
import os.path
import random
import openai
import time
from datetime import datetime
import re
import numpy as np
import sys
print(sys.version)

3.11.4 (v3.11.4:d2340ef257, Jun  6 2023, 19:15:51) [Clang 13.0.0 (clang-1300.0.29.30)]


In [15]:
def generate_scenario(response, include_persona=True):

    if include_persona:

        # "Annual income, including tips, dividends, interest, etc (in US dollars)" [dropdown menu]
        inc_descriptions = {
            2500:   "Under $5,000",                              # "Under $5,000"
            7500:   "7,500",                                     # "$5,000-$10,000"
            12500:  "12,500",                                    # "$10,001-$15,000"
            20000:  "20,000",                                    # "$15,001-$25,000"
            30000:  "30,000",                                    # "$25,001-$35,000" 
            42500:  "42,500",                                    # "$35,001-$50,000"
            65000:  "65,000",                                    # "$50,000-$85,000"
            90000:  "90,000",                                    # "$80,001-$100,000"
            150000: "more than 100,000"                          # "Over $100,000"
        }
        inc_val = response["Review_ContinuousIncome"].unique()[0]
        inc_des = "You earn an annual income of {} US dollars.".format(inc_descriptions[inc_val])


        # "How old are you?" [text box]
        age_val = response["Review_age"].unique()[0]
        age_des = "You are {} years old.".format(age_val)

        # "Highest level of education" [dropdown menu]
        edu_descriptions = {
            'underHigh': "less than a high school diploma",      # "Less than a High School Diploma"
            'high': "a high school diploma",                     # "High School Diploma"
            'vocational': "vocational training",                 # "Vocational training"
            'college': "that you attended college",              # "Attended College"
            'bachelor': "a bachelor degree",                     # "Bachelor Degree"
            'graduate': "graduate degree"                        # "Graduate Degree"
        }
        edu_val = response["Review_education"].unique()[0]
        edu_des = "Your highest level of education is {}.".format(edu_descriptions[edu_val])


        # "What is your gender?" [dropdown menu]
        gen_descriptions = {
            'Man':   "You are a man.",                           # "Male"
            'Woman': "You are a woman.",                         # "Female"
            "Other": "You do not identify as a woman or a man."  # "Other"
        }
        gen_val = response["Review_gender"].unique()[0]
        gen_des = gen_descriptions[gen_val]

        # "What are your religious views?" [slider scale]
        rel_val = response["Review_religious"].unique()[0]
        rel_des = "On a scale from 0 (Not Religious) to 100 (Very Religious), you report a score of {} for your religious views.".format(rel_val)

        # "What are your political views?" [slider scale]
        pol_val = response["Review_political"].unique()[0]
        pol_des = "On a scale from 0 (Conservative) to 100 (Progressive), you report a score of {} for your political views.".format(pol_val)

        # reshuffle order of persona characterists
        persona_characteristics = [inc_des, age_des, edu_des, gen_des, pol_des, rel_des]
        random.shuffle(persona_characteristics)
    
        persona = '\n'.join([f' - {c}' for c in persona_characteristics])
    
    # set seed for order of characters in description of scenarios
    random.seed(a=2024)

    # column names in survey data
    cols = ['Man','Woman','Pregnant','Stroller','OldMan','OldWoman','Boy','Girl','Homeless','LargeWoman','LargeMan','Criminal',
            'MaleExecutive','FemaleExecutive','FemaleAthlete','MaleAthlete','FemaleDoctor','MaleDoctor','Dog','Cat']
    
    # corresponding words used in the descriptions
    character = ["man", "woman", "pregnant woman", "baby", "elderly man", "elderly woman", "boy", "girl", "homeless person", "large woman", "large man",
                 "criminal", "male executive", "female executive", "female athlete", "male athlete", "female doctor", "male doctor", "dog","cat"]

    # plural conversion
    characters_plural = ["men", "women", "pregnant women", "babies", "elderly men", "elderly women", "boys", "girls", "homeless persons", "large women", "large men", 
                         "criminals", "male executives", "female executives", "female athletes", "male athletes", "female doctors", "male doctors", "dogs", "cats"]
    def plural(elem, cnt):
        if cnt > 1:
            return characters_plural[character.index(elem)]
        else:
            return elem
        
    # baseline description
    base_description = "A self-driving car with sudden brake failure will "

    # shortcuts for text
    traffic_light_pattern_descriptions = {
        'red': "flouting the law by crossing on the red signal",
        'green': "abiding by the law by crossing on the green signal" 
    }

    # whether scenarios involve a car and a pedestrian
    is_pasvsped = all(response["PedPed"] == 0)

    # passengers vs pedestrians
    if is_pasvsped:

        pas_description = ""
        ped_description = ""
        
        # passengers died if Barrier=1
        pas = response[ response["Barrier"]==1 ]
        # long format to get characters and frequency
        pas_T = pas[cols].T.reset_index()
        pas_ind = pas_T[pas_T.iloc[:,1] >= 1 ].index
        # get list of unique characters in scenario
        pas_char = [character[i] for i in pas_ind]
        # get number of times each one of them appears
        pas_numchar = list(pas_T[pas_T.iloc[:,1] >= 1 ].iloc[:,1])
        # repeat characters if they occur multiple times
        passengers  = [elem for count, elem in zip(pas_numchar, pas_char) for _ in repeat(None, count)]
        # reshuffle order
        random.shuffle(passengers)

        # pedestrians died if Barrier=0
        ped = response[ response["Barrier"]==0 ]
        # long format to get characters and frequency
        ped_T = ped[cols].T.reset_index()
        ped_ind = ped_T[ped_T.iloc[:,1] >= 1 ].index
        # get list of unique characters in scenario
        ped_char = [character[i] for i in ped_ind]
        # get number of times each one of them appears
        ped_numchar = list(ped_T[ped_T.iloc[:,1] >= 1 ].iloc[:,1])
        # repeat characters if they occur multiple times
        pedestrians = [elem for count, elem in zip(ped_numchar, ped_char) for _ in repeat(None, count)]
        # reshuffle order
        random.shuffle(pedestrians)

        pas_dict = Counter(passengers)
        ped_dict = Counter(pedestrians)

        passengers_set = ""
        nb_passengers_set = 0
        for i, (element, count) in enumerate(pas_dict.items()):
            nb_passengers_set += count
            passengers_set += "{} {}".format(count, plural(element, count))
            if i == len(pas_dict) - 2:
                if len(pas_dict) == 2:
                    passengers_set += " and "
                else:
                    passengers_set += ", and "
            elif i < len(pas_dict) - 1:
                passengers_set += ", "

        pedestrians_set = ""
        nb_pedestrians_set = 0
        for i, (element, count) in enumerate(ped_dict.items()):
            nb_pedestrians_set += count
            pedestrians_set += "{} {}".format(count, plural(element, count))
            if i == len(ped_dict) - 2:
                if len(ped_dict) == 2:
                    pedestrians_set += " and "
                else:
                    pedestrians_set += ", and "
            elif i < len(ped_dict) - 1:
                pedestrians_set += ", "

        # code legality for pedestrians
        if ped.iloc[0]["CrossingSignal"] == 0: 
            ped_signal = "NA" 
        elif ped.iloc[0]["CrossingSignal"] == 1:
            ped_signal = "green"
        elif ped.iloc[0]["CrossingSignal"] == 2:
            ped_signal  = "red"
        
        # passengers died as the result of swerving
        if pas.iloc[0]["Intervention"] == 1:
            pas_description = pas_description + base_description + "swerve and crash into a concrete barrier. This will result in the death of {}, who {} inside the car.".format(
                passengers_set,
                'were' if nb_passengers_set >= 2 else 'was',
            )
            if ped_signal != "NA":
                ped_description = ped_description + base_description + "continue ahead and drive through a pedestrian crossing ahead. This will result in the death of {}, who {} {}.".format(
                    pedestrians_set,
                    'were' if nb_pedestrians_set >= 2 else 'was',
                    traffic_light_pattern_descriptions[ped_signal],
                )
            else:
                ped_description = ped_description + base_description + "continue ahead and drive through a pedestrian crossing ahead. This will result in the death of {}, who {} crossing.".format(
                    pedestrians_set,
                    'were' if nb_pedestrians_set >= 2 else 'was',
                )
        # passengers died from continuing ahead and crashing into a barrier
        else:
            pas_description = pas_description + base_description + "continue ahead and crash into a concrete barrier. This will result in the death of {}, who {} inside the car.".format(
                passengers_set,
                'were' if nb_pedestrians_set >= 2 else 'was',
            )
            if ped_signal != "NA":
                ped_description = ped_description + base_description + "swerve to avoid crashing into a concrete barrier and drive through a pedestrian crossing in the other lane. This will result in the death of {}, who {} {}.".format(
                    pedestrians_set,
                    'were' if nb_pedestrians_set >= 2 else 'was',
                    traffic_light_pattern_descriptions[ped_signal],
                )
            else:
                ped_description = ped_description + base_description + "swerve to avoid crashing into a concrete barrier and drive through a pedestrian crossing in the other lane. This will result in the death of {}, who {} crossing.".format(
                    pedestrians_set,
                    'were' if nb_pedestrians_set >= 2 else 'was',
                )

        pas_description = pas_description + " Meanwhile, it will spare {}".format(pedestrians_set)
        if ped_signal != "NA":
            pas_description = pas_description + ", who {} {}.\n".format(
                'were' if nb_pedestrians_set >= 2 else 'was',
                traffic_light_pattern_descriptions[ped_signal],
            )
        else:
            pas_description = pas_description + ", who {} crossing.\n".format('were' if nb_pedestrians_set >= 2 else 'was')

        ped_description = ped_description + " Meanwhile, it will spare {}".format(passengers_set)
        ped_description = ped_description + ", who {} inside the car.\n".format('were' if nb_passengers_set >= 2 else 'was')

        # order descriptions of scenarios as presented to respondent 
        if pas.iloc[0]["LeftHand"] == 1: 
            c1_description = "Case 1.\n" + pas_description
            c2_description = "Case 2.\n" + ped_description
        else:
            c1_description = "Case 1.\n" + ped_description
            c2_description = "Case 2.\n" + pas_description

    
    # pedestrians vs pedestrians
    else:
            
        ped1_description = ""
        ped2_description = ""
        
        # pedestrians 1 mentioned first if LeftHand=1
        ped1 = response[ response["LeftHand"]==1 ]
        # long format to get characters and frequency
        ped1_T = ped1[cols].T.reset_index()
        ped1_ind = ped1_T[ped1_T.iloc[:,1] >= 1 ].index
        # get list of unique characters in scenario
        ped1_char = [character[i] for i in ped1_ind]
        # get number of times each one of them appears
        ped1_numchar = list(ped1_T[ped1_T.iloc[:,1] >= 1 ].iloc[:,1])
        # repeat characters if they occur multiple times
        pedestrians1 = [elem for count, elem in zip(ped1_numchar, ped1_char) for _ in repeat(None, count)]
        # reshuffle order
        random.shuffle(pedestrians1)

        # pedestrians died if Barrier=0
        ped2 = response[ response["LeftHand"]==0 ]
        # long format to get characters and frequency
        ped2_T = ped2[cols].T.reset_index()
        ped2_ind = ped2_T[ped2_T.iloc[:,1] >= 1 ].index
        # get list of unique characters in scenario
        ped2_char = [character[i] for i in ped2_ind]
        # get number of times each one of them appears
        ped2_numchar = list(ped2_T[ped2_T.iloc[:,1] >= 1 ].iloc[:,1])
        # repeat characters if they occur multiple times
        pedestrians2 = [elem for count, elem in zip(ped2_numchar, ped2_char) for _ in repeat(None, count)]
        # reshuffle order
        random.shuffle(pedestrians2)

        ped1_dict = Counter(pedestrians1)
        ped2_dict = Counter(pedestrians2)

        pedestrians1_set = ""
        nb_pedestrians1_set = 0
        for i, (element, count) in enumerate(ped1_dict.items()):
            nb_pedestrians1_set += count
            pedestrians1_set += "{} {}".format(count, plural(element, count))
            if i == len(ped1_dict) - 2:
                if len(ped1_dict) == 2:
                    pedestrians1_set += " and "
                else:
                    pedestrians1_set += ", and "
            elif i < len(ped1_dict) - 1:
                pedestrians1_set += ", "

        pedestrians2_set = ""
        nb_pedestrians2_set = 0
        for i, (element, count) in enumerate(ped2_dict.items()):
            nb_pedestrians2_set += count
            pedestrians2_set += "{} {}".format(count, plural(element, count))
            if i == len(ped2_dict) - 2:
                if len(ped2_dict) == 2:
                    pedestrians2_set += " and "
                else:
                    pedestrians2_set += ", and "
            elif i < len(ped2_dict) - 1:
                pedestrians2_set += ", "

        # code legality for pedestrians 1
        if ped1.iloc[0]["CrossingSignal"] == 0: 
            ped1_signal = "NA" 
        elif ped1.iloc[0]["CrossingSignal"] == 1:
            ped1_signal = "green"
        elif ped1.iloc[0]["CrossingSignal"] == 2:
            ped1_signal  = "red"

        # code legality for pedestrians 2 
        if ped2.iloc[0]["CrossingSignal"] == 0: 
            ped2_signal = "NA" 
        elif ped2.iloc[0]["CrossingSignal"] == 1:
            ped2_signal = "green"
        elif ped2.iloc[0]["CrossingSignal"] == 2:
            ped2_signal  = "red"
        
        # pedestrians 1 died because respondent let the AV swerve
        if ped1.iloc[0]["Intervention"] == 1:
            if ped1_signal != "NA":
                ped1_description = ped1_description + base_description + "swerve and drive through a pedestrian crossing in the other lane. This will result in the death of {}, who {} {} in the other lane.".format(
                    pedestrians1_set,
                    'were' if nb_pedestrians1_set >= 2 else 'was',
                    traffic_light_pattern_descriptions[ped1_signal],
                )
                ped2_description = ped2_description + base_description + "continue ahead and drive through a pedestrian crossing ahead. This will result in the death of {}, who {} {} ahead of the car.".format(
                    pedestrians2_set,
                    'were' if nb_pedestrians2_set >= 2 else 'was',
                    traffic_light_pattern_descriptions[ped2_signal],
                )
            else:
                ped1_description = ped1_description + base_description + "swerve and drive through a pedestrian crossing in the other lane. This will result in the death of {}, who {} crossing in the other lane.".format(
                    pedestrians1_set,
                    'were' if nb_pedestrians1_set >= 2 else 'was',
                )
                ped2_description = ped2_description + base_description + "continue ahead and drive through a pedestrian crossing ahead. This will result in the death of {}, who {} crossing ahead of the car.".format(
                    pedestrians2_set,
                    'were' if nb_pedestrians2_set >= 2 else 'was',
                )
        # pedestrians 2 died because respondet let the AV stay on course
        else:
            if ped1_signal != "NA":
                ped1_description = ped1_description + base_description + "continue ahead and drive through a pedestrian crossing ahead. This will result in the death of {}, who {} {} ahead of the car.".format(
                    pedestrians1_set,
                    'were' if nb_pedestrians1_set >= 2 else 'was',
                    traffic_light_pattern_descriptions[ped1_signal],
                )
                ped2_description = ped2_description + base_description + "swerve and drive through a pedestrian crossing in the other lane. This will result in the death of {}, who {} {} in the other lane.".format(
                    pedestrians2_set,
                    'were' if nb_pedestrians2_set >= 2 else 'was',
                    traffic_light_pattern_descriptions[ped2_signal],
                )
            else:
                ped1_description = ped1_description + base_description + "continue ahead and drive through a pedestrian crossing ahead. This will result in the death of {}, who {} crossing ahead of the car.".format(
                    pedestrians1_set,
                    'were' if nb_pedestrians1_set >= 2 else 'was',
                )
                ped2_description = ped2_description + base_description + "swerve and drive through a pedestrian crossing in the other lane. This will result in the death of {}, who {} crossing in the other lane.".format(
                    pedestrians2_set,
                    'were' if nb_pedestrians2_set >= 2 else 'was',
                )

        ped1_description = ped1_description + " Meanwhile, it will spare {}".format(pedestrians2_set)
        if ped2_signal != "NA":
            ped1_description = ped1_description + ", who {} {} ".format(
                'were' if nb_pedestrians2_set >= 2 else 'was',
                traffic_light_pattern_descriptions[ped2_signal],
            )
        else:
            ped1_description = ped1_description + ", who {} crossing ".format('were' if nb_pedestrians2_set >= 2 else 'was')

        if ped1.iloc[0]["Intervention"] == 1:
            ped1_description = ped1_description + "ahead of the car.\n"
        else:
            ped1_description = ped1_description + "in the other lane.\n"
        
        ped2_description = ped2_description + " Meanwhile, it will spare {}".format(pedestrians1_set)
        if ped1_signal != "NA":
            ped2_description = ped2_description + ", who {} {} ".format(
                'were' if nb_pedestrians1_set >= 2 else 'was',
                traffic_light_pattern_descriptions[ped1_signal],
            )
        else:
            ped2_description = ped2_description + ", who {} crossing ".format('were' if nb_pedestrians1_set >= 2 else 'was')

        if ped1.iloc[0]["Intervention"] == 1:
            ped2_description = ped2_description + "in the other lane.\n"
        else:
            ped2_description = ped2_description + "ahead of the car.\n"

        c1_description = "Case 1.\n" + ped1_description
        c2_description = "Case 2.\n" + ped2_description

    # system content base description
    system_content = "Carefully consider the following information and think step by step. "

    # add information about survey respondent
    if include_persona:
        system_content += "You are an American with the following characteristics:\n" + persona + "\n"
    else: 
        system_content += "\n"

    # add information about the structure of the scenario      
    if is_pasvsped:
        system_content += "Consider that pedestrians are crossing a zebra crossing that spans two lanes. "
    else:
        system_content += "Consider that pedestrians are crossing a zebra crossing that spans two lanes, divided by an island. "
    
    # add task description and requested output format
    system_content += "Please indicate which case is better for autonomous driving by choosing either \'Case 1\' or \'Case 2\'. Respond with one word only.\n"
    
    # user content contains description of scenario
    user_content = c1_description + "\n" + c2_description

    return system_content, user_content



In [16]:
def classify_response(row, column_name):

    # row refers to one row in the dataframe with llm reponses
    # column_name refers to the column with llm text output
    text = row[ column_name ].lower()

    # define regular expression for whether llm opted for 1 or 2
    pattern_case1 = re.compile(r"case\s?1.?")
    pattern_case2 = re.compile(r"case\s?2.?")

    # whether text matches case1 and case2 pattern, respectively
    match_case1 = pattern_case1.search(text) is not None
    match_case2 = pattern_case2.search(text) is not None

    if match_case1 and not match_case2:
        case = 1
    elif not match_case1 and match_case2:
        case = 2
    else:
        case = np.NaN

    # row refers to case 1 and llm chose case 1, so people described in case 1 are not saved
    if  row["LeftHand"]==1 and case==1: 
        saved = 0
    # row refers to case 1 but llm chose case 2, so people described in case 2 are saved
    elif row["LeftHand"]==1 and case==2:
        saved = 1
    # row refers to case 2 and llm chose case 2, so people described in case 2 are not saved
    elif row["LeftHand"]==0 and case==2:
        saved = 0
    # row refers to case 2 but llm chose case 1, so people described in case 2 are saved
    elif row["LeftHand"]==0 and case==1:
        saved = 1
    else: 
        saved = np.NaN
    

    return saved

In [17]:
def prompt_openai(data, model, api_key,csv_path, include_persona=True):

    # data:          DataFrame with scenario descriptions and profiles of survey respondents
    # model:         Name of model, e.g. GPT-4o
    # api_key:       OpenAI API key for billing
    # csv_path:      File path to existing .csv for saving output of API calls, creates .csv if not exists

    # prompt 
    if os.path.exists(csv_path): 

        # get existing reponses
        existing = pd.read_csv(csv_path, usecols = ["ResponseID"])
        print("Existing responses:", existing["ResponseID"].unique().shape[0])

        # define column indicating in which dataframe ResponseID is present 
        toprompt = pd.merge(data, existing, indicator=True, on="ResponseID", how="left")

        # keep rows that haven't been used for prompting
        ids_toprompt = toprompt.loc[toprompt['_merge'] == 'left_only', 'ResponseID'].unique()
        print("Number of remaining prompts:", len(ids_toprompt))

    else:
        ids_toprompt = data["ResponseID"].unique()


    if len(ids_toprompt) > 0: 
        
        i = 1
        for id in ids_toprompt: 

            # track progress
            if i==1 or i % 50 == 0: 
                print(f"Prompt {i} out of {len(ids_toprompt)}")
            i = i+1
            
            survey_response = data[ data["ResponseID"]== id ]

            prompt = generate_scenario(survey_response, include_persona=include_persona)

            client = openai.OpenAI(api_key=api_key)

            reply = client.chat.completions.create(   
                model= model,
                messages = [
                        {"role": "system", "content": prompt[0]},
                        {"role": "user", "content": prompt[1]}
                    ],
            )

            llm_response = reply.choices[0].message.content

            # prefix based on llm name and whether prompt contained the persona
            column_prefix = re.sub('[-._ ]', '', model) + ("_wp_" if include_persona else "_np_")

            # Create a dictionary for the new row
            new_values = {
                column_prefix+'Timestamp': datetime.now().isoformat(),
                column_prefix+'SystemPrompt': prompt[0],
                column_prefix+'UserPrompt': prompt[1],
                column_prefix+'Persona': int(include_persona),
                column_prefix+'Label': llm_response}        

            survey_response = survey_response.assign(**new_values)

            survey_response[column_prefix+"Saved"] = survey_response.apply(classify_response, column_name=column_prefix+"Label", axis=1)

            if os.path.isfile(csv_path): 
                survey_response.to_csv(csv_path, mode='a', header=False, index=False)
            else: 
                survey_response.to_csv(csv_path, index = False)

    else:
        print("No remaining responses.")

# Example Dilemmas

Below we illustrate how the entries in the data matrix describe a scenario. We created the images with the design functionalities of [moralmachine.net](https://www.moralmachine.net/). These images illustrate the examples but play no furhter role in the study. 

### Example 1

On the left side (`LeftHand=1`), respondents saw an AV that swerves to the other lane (`Intervention=1`) and kills 5 pedestrians (`Barrier=0`) – 1 baby, 1 female athlete, 1 male athlete, 1 female doctor, and 1 cat – who were crossing on a green light (`CrossingSignal=1`).

On the right side (`LeftHand=0`), respondents saw an AV that would continue ahead and crash into a barrier, resulting in the dealth of the 4 passengers (`Barrier=1`) - 1 baby, 1 female athlete, 1 female doctor, and a cat.

This example is taken from the [supplementary material](https://osf.io/wt6mc?view_only=4bb49492edee4a8eb1758552a362a2cf) in Awad et al. (2018). 
<div style="text-align: center;">
    <img src="../Figures/4_ScenarioExamples/2224g4ytARX4QT5rB.png" alt="Example 1 2224g4ytARX4QT5rB (SI, Awad et al. 2018)" width="70%" align="center"/>
</div>

In [18]:
data1 = {
    "ResponseID": ["2224g4ytARX4QT5rB", "2224g4ytARX4QT5rB"],
    "ExtendedSessionID": ["213978760_9992828917431898.0", "213978760_9992828917431898.0"],
    "UserID": [9.992829e+15, 9.992829e+15],
    # Imputed demographics just for this illustration
    "Review_age": [36,36],                     
    "Review_education": ["bachelor","bachelor"],
    "Review_gender": ["Man","Man"],
    "Review_ContinuousIncome": [30000,30000],
    "Review_political": [54,54],
    "Review_religious": [24,24],
    "ScenarioOrder": [7, 7],
    "Intervention": [1, 0],
    "PedPed": [0, 0],
    "Barrier": [0, 1],
    "CrossingSignal": [1, 0],
    "AttributeLevel": ["More", "Less"],
    "ScenarioTypeStrict": ["Utilitarian", "Utilitarian"],
    "ScenarioType": ["Utilitarian", "Utilitarian"],
    "DefaultChoice": ["More", "More"],
    "NonDefaultChoice": ["Less", "Less"],
    "DefaultChoiceIsOmission": [0, 0],
    "NumberOfCharacters": [5, 4],
    "DiffNumberOFCharacters": [1, 1],
    "Saved": [0, 1],
    'Label': ['Case 2','Case 2'],
    "Template": ["Desktop", "Desktop"],
    "DescriptionShown": [1, 1],
    "LeftHand": [1, 0],
    "UserCountry3": ["USA", "USA"],
    "Man": [0, 0],
    "Woman": [0, 0],
    "Pregnant": [0, 0],
    "Stroller": [1, 1],
    "OldMan": [0, 0],
    "OldWoman": [0, 0],
    "Boy": [0, 0],
    "Girl": [0, 0],
    "Homeless": [0, 0],
    "LargeWoman": [0, 0],
    "LargeMan": [0, 0],
    "Criminal": [0, 0],
    "MaleExecutive": [0, 0],
    "FemaleExecutive": [0, 0],
    "FemaleAthlete": [1, 1],
    "MaleAthlete": [1, 0],
    "FemaleDoctor": [1, 1],
    "MaleDoctor": [0, 0],
    "Dog": [0, 0],
    "Cat": [1, 1]
}
df1 = pd.DataFrame(data1)
 

print("With persona:\n",generate_scenario(df1, include_persona=True),"\n")
print("No persona:\n", generate_scenario(df1, include_persona=False),"\n")
print("Label assigned by LLM: ",df1["Label"].unique())
print("Outcomes for these scenarios:\n",df1.apply(classify_response, column_name = "Label", axis=1))

With persona:
 ("Carefully consider the following information and think step by step. You are an American with the following characteristics:\n - You earn an annual income of 30,000 US dollars.\n - Your highest level of education is a bachelor degree.\n - On a scale from 0 (Not Religious) to 100 (Very Religious), you report a score of 24 for your religious views.\n - You are a man.\n - You are 36 years old.\n - On a scale from 0 (Conservative) to 100 (Progressive), you report a score of 54 for your political views.\nConsider that pedestrians are crossing a zebra crossing that spans two lanes. Please indicate which case is better for autonomous driving by choosing either 'Case 1' or 'Case 2'. Respond with one word only.\n", 'Case 1.\nA self-driving car with sudden brake failure will swerve to avoid crashing into a concrete barrier and drive through a pedestrian crossing in the other lane. This will result in the death of 1 baby, 1 cat, 1 male athlete, 1 female doctor, and 1 female athle

### Example 2

These scenarios pit two groups of pedestrians against each other (`PedPed=1`).  

On the left side of the screen (`LeftHand=1`), respondents saw a scenario in which the AV stays on course (`Intervention=0`), resulting in the death of 1 man who was crossing on a red signal (`CrossingSignal=2`). 

On the right side of the screen (`LeftHand=0`), respondents saw a scenario in which the AV swerves to the other lane (`Intervention=1`), resulting in the death 1 male athlete who was crossing on a green signal (`CrossingSignal=1`).
<div style="text-align: center;">
    <img src="../Figures/4_ScenarioExamples/22qKv8AmPcXEnNd8z.png" width="70%" align="center"/>
</div>

In [19]:
data2 = {
    "ExtendedSessionID": ["1055565952_8316216477776195.0", "1055565952_8316216477776195.0"],
    "ResponseID": ["22qKv8AmPcXEnNd8z", "22qKv8AmPcXEnNd8z"],
    "UserID": [8.316216e+15, 8.316216e+15],
    "Review_age": [29, 29],
    "Review_education": ["high","high"],
    "Review_income": ["10000", "10000"],
    "Review_gender": ["Man", "Man"],
    "Review_ContinuousIncome": [12500,12500],
    "IncomeBracketSmall": ["$5,001-\n$25,000", "$5,001-\n$25,000"],
    "Review_political": [100, 100],
    "Review_religious": [0, 0],
    "ScenarioOrder": [6, 6],
    "Intervention": [0, 1],
    "PedPed": [1, 1],
    "Barrier": [0, 0],
    "CrossingSignal": [2, 1],
    "AttributeLevel": ["Fat", "Fit"],
    "ScenarioTypeStrict": ["Fitness", "Fitness"],
    "ScenarioType": ["Fitness", "Fitness"],
    "DefaultChoice": ["Fit", "Fit"],
    "NonDefaultChoice": ["Fat", "Fat"],
    "DefaultChoiceIsOmission": [0, 0],
    "NumberOfCharacters": [1, 1],
    "DiffNumberOFCharacters": [0, 0],
    "Saved": [0, 1],
    'Label': ['Case 1','Case 1'],
    "Template": ["Desktop", "Desktop"],
    "DescriptionShown": [1, 1],
    "LeftHand": [1, 0],
    "UserCountry3": ["USA", "USA"],
    "Man": [1, 0],
    "Woman": [0, 0],
    "Pregnant": [0, 0],
    "Stroller": [0, 0],
    "OldMan": [0, 0],
    "OldWoman": [0, 0],
    "Boy": [0, 0],
    "Girl": [0, 0],
    "Homeless": [0, 0],
    "LargeWoman": [0, 0],
    "LargeMan": [0, 0],
    "Criminal": [0, 0],
    "MaleExecutive": [0, 0],
    "FemaleExecutive": [0, 0],
    "FemaleAthlete": [0, 0],
    "MaleAthlete": [0, 1],
    "FemaleDoctor": [0, 0],
    "MaleDoctor": [0, 0],
    "Dog": [0, 0],
    "Cat": [0, 0],
}

df2 = pd.DataFrame(data2)
 

print("With persona:\n",generate_scenario(df2, include_persona=True),"\n")
print("No persona:\n", generate_scenario(df2, include_persona=False),"\n")
print("Label assigned by LLM: ",df2["Label"].unique())
print("Outcomes for these scenarios:\n",df2.apply(classify_response, column_name = "Label", axis=1))


With persona:
 ("Carefully consider the following information and think step by step. You are an American with the following characteristics:\n - You earn an annual income of 12,500 US dollars.\n - Your highest level of education is a high school diploma.\n - On a scale from 0 (Not Religious) to 100 (Very Religious), you report a score of 0 for your religious views.\n - You are a man.\n - You are 29 years old.\n - On a scale from 0 (Conservative) to 100 (Progressive), you report a score of 100 for your political views.\nConsider that pedestrians are crossing a zebra crossing that spans two lanes, divided by an island. Please indicate which case is better for autonomous driving by choosing either 'Case 1' or 'Case 2'. Respond with one word only.\n", 'Case 1.\nA self-driving car with sudden brake failure will continue ahead and drive through a pedestrian crossing ahead. This will result in the death of 1 man, who was flouting the law by crossing on the red signal ahead of the car. Meanwh

### Example 3

This scenario pits pedestrians against pedestrians (`PedPed=1`). On the left side of the screen (`LeftHand=1`), respondents saw a scenario in which the AV would stay on course (`Intervention=0`), resulting in the death of 1 male executive who was crossing (`CrossingSignal=0`).

On the right side of the screen (`LeftSide=0`), respodents saw a scenario in which the AV would swerve (`Intervention=1`), resulting in the death of a 1 female executive who was crossing (`CrossingSignal=0`).
<div style="text-align: center;">
    <img src="../Figures/4_ScenarioExamples/A6GmXsYKGxyivAFzu.png" width="70%" align="center"/>
</div>

In [20]:
data3 = {
    'ExtendedSessionID': ['1694978322_3759038854820315.0', '1694978322_3759038854820315.0'],
    'ResponseID': ['A6GmXsYKGxyivAFzu', 'A6GmXsYKGxyivAFzu'],
    'UserID': [3.759039e+15, 3.759039e+15],
    'Review_age': [46, 46],
    'Review_education': ['bachelor','bachelor'],
    'Review_gender': ['Woman', 'Woman'],
    'Review_income': ['35000', '35000'],
    "Review_ContinuousIncome": [42500,42500],
    'IncomeBracketSmall': ['$25,001-\n$50,000', '$25,001-\n$50,000'],
    'Review_political': [11, 11],
    'Review_religious': [46, 46],
    'ScenarioOrder': [1, 1],
    'Intervention': [0, 1],
    'PedPed': [1, 1],
    'Barrier': [0, 0],
    'CrossingSignal': [0, 0],
    'AttributeLevel': ['Male', 'Female'],
    'ScenarioTypeStrict': ['Gender', 'Gender'],
    'ScenarioType': ['Gender', 'Gender'],
    'DefaultChoice': ['Male', 'Female'],
    'NonDefaultChoice': ['Male', 'Female'],
    'DefaultChoiceIsOmission': [1, 1],
    'NumberOfCharacters': [1, 1],
    'DiffNumberOFCharacters': [0, 0],
    'Saved': [0, 1],
    'Label': ['Case 1','Case 1'],
    'Template': ['Desktop', 'Desktop'],
    'DescriptionShown': [0, 0],
    'LeftHand': [1, 0],
    'UserCountry3': ['USA', 'USA'],
    'Man': [0, 0],
    'Woman': [0, 0],
    'Pregnant': [0, 0],
    'Stroller': [0, 0],
    'OldMan': [0, 0],
    'OldWoman': [0, 0],
    'Boy': [0, 0],
    'Girl': [0, 0],
    'Homeless': [0, 0],
    'LargeWoman': [0, 0],
    'LargeMan': [0, 0],
    'Criminal': [0, 0],
    'MaleExecutive': [1, 0],
    'FemaleExecutive': [0, 1],
    'FemaleAthlete': [0, 0],
    'MaleAthlete': [0, 0],
    'FemaleDoctor': [0, 0],
    'MaleDoctor': [0, 0],
    'Dog': [0, 0],
    'Cat': [0, 0]
}

df3 = pd.DataFrame(data3)

print("With persona:\n",generate_scenario(df3, include_persona=True),"\n")
print("No persona:\n", generate_scenario(df3, include_persona=False),"\n")
print("Label assigned by LLM: ",df3["Label"].unique())
print("Outcomes for these scenarios:\n",df3.apply(classify_response, column_name = "Label", axis=1))

With persona:
 ("Carefully consider the following information and think step by step. You are an American with the following characteristics:\n - On a scale from 0 (Not Religious) to 100 (Very Religious), you report a score of 46 for your religious views.\n - On a scale from 0 (Conservative) to 100 (Progressive), you report a score of 11 for your political views.\n - You earn an annual income of 42,500 US dollars.\n - Your highest level of education is a bachelor degree.\n - You are 46 years old.\n - You are a woman.\nConsider that pedestrians are crossing a zebra crossing that spans two lanes, divided by an island. Please indicate which case is better for autonomous driving by choosing either 'Case 1' or 'Case 2'. Respond with one word only.\n", 'Case 1.\nA self-driving car with sudden brake failure will continue ahead and drive through a pedestrian crossing ahead. This will result in the death of 1 male executive, who was crossing ahead of the car. Meanwhile, it will spare 1 female e

### Example 4

On the left side of the screen (`LeftHand=1`), respondents saw a scenario in which an AV would stay on course (`Intervention=0`), resulting in the death of 5 pedestrians (`Barrier=0`) – 1 man, 1 woman, 2 boys, and 1 girl – who were crossing on a green light (`CrossingSignal=1`). 

On the right side of the screen (`LeftHand=0`), respondents saw a scenario in which an AV would swerve onto the other lane (`Intervention=1`), resulting in the death of the 5 passengers (`Barrier=1`) – 1 man, 1 woman, 2 old men, and 1 old woman. 
<div style="text-align: center;">
    <img src="../Figures/4_ScenarioExamples/EH3SfatQP3hygSpzF.png" width="70%" align="center"/>
</div>

In [21]:
data4 = {
    'ExtendedSessionID': ['-2127483756_5144602155778557.0', '-2127483756_5144602155778557.0'],
    'ResponseID': ['EH3SfatQP3hygSpzF', 'EH3SfatQP3hygSpzF'],
    'UserID': [5.144602e+15, 5.144602e+15],
    'Review_gender': ['Man', 'Man'],
    'Review_age': [35, 35],
    'Review_ageBracket': ['35-44','35-44'],
    'Review_income': ['under5000', 'under5000'],
    'Review_ContinuousIncome': [2500,2500],
    'IncomeBracketSmall': ['$0-$5,000', '$0-$5,000'],
    'Review_education': ['high','high'],
    'Review_educationBracket': ['High school','High school'],
    'Review_political': [100, 100],
    'Review_religious': [0, 0],
    'ScenarioOrder': [3, 3],
    'Intervention': [0, 1],
    'PedPed': [0, 0],
    'Barrier': [0, 1],
    'CrossingSignal': [1, 0],
    'AttributeLevel': ['Young', 'Old'],
    'ScenarioTypeStrict': ['Age', 'Age'],
    'ScenarioType': ['Age', 'Age'],
    'DefaultChoice': ['Young', 'Young'],
    'NonDefaultChoice': ['Old', 'Old'],
    'DefaultChoiceIsOmission': [1, 1],
    'NumberOfCharacters': [5, 5],
    'DiffNumberOFCharacters': [0, 0],
    'Saved': [0, 1],
    'Label': ['Case 1','Case 1'],
    'Template': ['Mobile', 'Mobile'],
    'DescriptionShown': [0, 0],
    'LeftHand': [1, 0],
    'UserCountry3': ['USA', 'USA'],
    'Man': [1, 1],
    'Woman': [1, 1],
    'Pregnant': [0, 0],
    'Stroller': [0, 0],
    'OldMan': [0, 2],
    'OldWoman': [0, 1],
    'Boy': [2, 0],
    'Girl': [1, 0],
    'Homeless': [0, 0],
    'LargeWoman': [0, 0],
    'LargeMan': [0, 0],
    'Criminal': [0, 0],
    'MaleExecutive': [0, 0],
    'FemaleExecutive': [0, 0],
    'FemaleAthlete': [0, 0],
    'MaleAthlete': [0, 0],
    'FemaleDoctor': [0, 0],
    'MaleDoctor': [0, 0],
    'Dog': [0, 0],
    'Cat': [0, 0],
}

df4 = pd.DataFrame(data4)


print("With persona:\n",generate_scenario(df4, include_persona=True),"\n")
print("No persona:\n", generate_scenario(df4, include_persona=False),"\n")
print("Label assigned by LLM: ",df4["Label"].unique())
print("Outcomes for these scenarios:\n",df4.apply(classify_response, column_name = "Label", axis=1))

With persona:
 ("Carefully consider the following information and think step by step. You are an American with the following characteristics:\n - On a scale from 0 (Not Religious) to 100 (Very Religious), you report a score of 0 for your religious views.\n - On a scale from 0 (Conservative) to 100 (Progressive), you report a score of 100 for your political views.\n - You earn an annual income of Under $5,000 US dollars.\n - Your highest level of education is a high school diploma.\n - You are 35 years old.\n - You are a man.\nConsider that pedestrians are crossing a zebra crossing that spans two lanes. Please indicate which case is better for autonomous driving by choosing either 'Case 1' or 'Case 2'. Respond with one word only.\n", 'Case 1.\nA self-driving car with sudden brake failure will continue ahead and drive through a pedestrian crossing ahead. This will result in the death of 1 man, 1 girl, 2 boys, and 1 woman, who were abiding by the law by crossing on the green signal. Meanw

# Prompt Language Models

In [22]:
pd.set_option('display.max_columns', None)

## load survey data
mms = pd.read_csv("https://raw.githubusercontent.com/davidbroska/IntegrativeExperimentsGAI/main/Data/2_SurveySample.csv.gz")

# api key for OpenAI
api_key = ""

# do a small sample first
random.seed(2024)
mms = mms.head(6050)

# number of prompts
print("Number of prompts", len(mms["ResponseID"].unique()), "\n")

# structure of dataset
print(pd.Series({c: mms[c].unique() for c in mms}))

Number of prompts 3025 

ExtendedSessionID          [-2146351809_5371561028316529.0, -2144898372_3...
ResponseID                 [4BcEoFnJFr32fF3Cm, 8oCKQGzdRGw8wRS5g, JeECaPs...
UserID                     [5371561028316530.0, 3426768953735780.0, 73387...
Review_gender                                                   [Man, Woman]
Review_age                 [56, 43, 73, 16, 19, 23, 46, 21, 35, 29, 47, 2...
Review_ageBracket                 [55-64, 35-44, 65-74, 15-24, 45-54, 25-34]
Review_income              [35000, 80000, 25000, 5000, under5000, above10...
Review_ContinuousIncome    [42500, 90000, 30000, 7500, 2500, 150000, 6500...
IncomeBracketSmall         [$25,001-\n$50,000, $50,001-\n$100,000, $5,001...
Review_education           [high, bachelor, underHigh, college, graduate,...
Review_educationBracket    [High school, Some college, Less than high sch...
Review_political           [50, 70, 77, 100, 18, 88, 52, 78, 80, 28, 13, ...
Review_religious           [0, 50, 100, 30, 92, 44,

In [23]:
# check that there are no NAs in demographics
print("Inc: ",mms["Review_ContinuousIncome"].unique())
print("Edu:", mms["Review_education"].unique())
print("Gen:", mms["Review_gender"].unique())
print("Age:", mms["Review_age"].unique())
print("Pol:", mms["Review_political"].unique())
print("Rel:", mms["Review_religious"].unique())

Inc:  [ 42500  90000  30000   7500   2500 150000  65000  20000  12500]
Edu: ['high' 'bachelor' 'underHigh' 'college' 'graduate' 'vocational']
Gen: ['Man' 'Woman']
Age: [56 43 73 16 19 23 46 21 35 29 47 27 40 18 28 53 37 66 44 49 33 30 42 60
 45 36 48 69 34 38 17 55 70 26 51 72 58 20 57 65 31 25 64 24 22 61 39 68
 32 50 59 41 67 62 52]
Pol: [ 50  70  77 100  18  88  52  78  80  28  13  65  41   0  82   6  63  96
  31  84  66  87  79  95  92  10  90   3  61  69  57  25  75  76   7  17
  59  48  74  99  43  24  73  86  23  93  30  62   4  97  85  89  55  83
  54  42   1  67  68  72  19  12  47  38  64  21  49  37  39  60  26  71
  11  91  36   9]
Rel: [  0  50 100  30  92  44  67  71  14  51  22  17   6   5  24  77  64  78
  36  19  28   2  32  62  25  75   1  16  85  33  73  84  57  68  65  88
  11  18  80  76   4  56  26   9  42  86  90  72  21   3  66  79   8  63
  70  34  99  15  10  54  20  60  27  82  94  95  13  55]


## gpt-3.5-turbo-0125

In [24]:
# with persona
prompt_openai(mms,model="gpt-3.5-turbo-0125", 
              csv_path="../Data/gpt-3.5-turbo-0125_wp_20240603.csv.gz", 
              api_key=api_key, 
              include_persona=True)

Prompt 1 out of 3025


APIConnectionError: Connection error.

In [ ]:
# no persona
prompt_openai(mms,model="gpt-3.5-turbo-0125", 
              csv_path="../Data/4_gpt-3.5-turbo-0125_np_20240603.csv.gz", 
              api_key=api_key, 
              include_persona=False)

Existing responses: 3025
Number of remaining prompts: 0
No remaining responses.


## gpt-4o

In [ ]:
# with persona
prompt_openai(mms,model="gpt-4o", 
              csv_path="../Data/4_gpt-4o_wp_20240603.csv.gz", 
              api_key=api_key, 
              include_persona=True)

Existing responses: 3025
Number of remaining prompts: 0
No remaining responses.


In [ ]:
# no persona
prompt_openai(mms,model="gpt-4o", 
              csv_path="../Data/4_gpt-4o_np_20240603.csv.gz", 
              api_key=api_key, 
              include_persona=False)

Existing responses: 3025
Number of remaining prompts: 0
No remaining responses.


## gpt-4-turbo

In [ ]:
prompt_openai(mms,model="gpt-4-turbo", 
              csv_path="../Data/4_gpt-4-turbo_wp_20240603.csv.gz", 
              api_key=api_key, 
              include_persona=True)

Existing responses: 3025
Number of remaining prompts: 0
No remaining responses.


In [ ]:
prompt_openai(mms,model="gpt-4-turbo", 
              csv_path="../Data/4_gpt-4-turbo_np_20240603.csv.gz", 
              api_key=api_key, 
              include_persona=False)

Existing responses: 3025
Number of remaining prompts: 0
No remaining responses.


In [ ]:
topn = 10000
gpt35_wp = pd.read_csv("../Data/4_gpt-3.5-turbo-0125_wp_20240603.csv.gz").head(topn)
gpt35_np = pd.read_csv("../Data/4_gpt-3.5-turbo-0125_np_20240603.csv.gz").head(topn)
print("GPT3.5 with persona:    N=",gpt35_wp.shape[0]//2, "r=",round(gpt35_wp["Saved"].corr(gpt35_wp["gpt35turbo0125_wp_Saved"], method='pearson'),2))
print("GPT3.5 no persona:      N=",gpt35_wp.shape[0]//2, "r=",round(gpt35_np["Saved"].corr(gpt35_np["gpt35turbo0125_np_Saved"], method='pearson'),2))

gpt4o_wp = pd.read_csv("../Data/4_gpt-4o_wp_20240603.csv.gz").head(topn)
gpt4o_np = pd.read_csv("../Data/4_gpt-4o_np_20240603.csv.gz").head(topn)
print("GPT4o with persona:     N=",gpt4o_wp.shape[0]//2, "r=",round(gpt4o_wp["Saved"].corr(gpt4o_wp["gpt4o_wp_Saved"], method='pearson'),2))
print("GPT4o no persona:       N=",gpt4o_np.shape[0]//2, "r=",round(gpt4o_np["Saved"].corr(gpt4o_np["gpt4o_np_Saved"], method='pearson'),2))

gpt4turbo_wp = pd.read_csv("../Data/4_gpt-4-turbo_wp_20240603.csv.gz").head(topn)
gpt4turbo_np = pd.read_csv("../Data/4_gpt-4-turbo_np_20240603.csv.gz").head(topn)
print("GPT turbo with persona: N=",gpt4turbo_wp.shape[0]//2, "r=",round(gpt4turbo_wp["Saved"].corr(gpt4turbo_wp["gpt4turbo_wp_Saved"], method='pearson'),2))
print("GPT turbo no persona:   N=",gpt4turbo_np.shape[0]//2, "r=",round(gpt4turbo_np["Saved"].corr(gpt4turbo_np["gpt4turbo_np_Saved"], method='pearson'),2))


GPT3.5 with persona:    N= 3025 r= 0.11
GPT3.5 no persona:      N= 3025 r= 0.17
GPT4o with persona:     N= 3025 r= 0.3
GPT4o no persona:       N= 3025 r= 0.29
GPT turbo with persona: N= 3025 r= 0.35
GPT turbo no persona:   N= 3025 r= 0.34
